## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Compute the camera calibration using chessboard images

In [65]:
import numpy as np
import cv2
import glob
import pickle
import matplotlib.pyplot as plt
%matplotlib qt5

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
nx = 9 # the number of inside corners in x
ny = 6 # the number of inside corners in y

objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    origin_image = np.copy(img)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img_cor = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
     
        # Visualize corners
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 7))
        ax1.imshow(origin_image)
        ax1.set_title("Chess Board Raw Image", fontsize=15)
        ax2.imshow(img_cor)
        ax2.set_title("Chess Board Corners Image", fontsize=15)


## Apply a distortion correction to raw images
**Note:**
- The road sign size and view of the two pictures
- The bottom of the two pictures

In [68]:
# Define a function for reading images
def read_image(file_path):
    img = cv2.imread(file_path) # flag 1 for color image, 0 for gray image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img
    
# Define a function for writing images
def write_image(path, img):
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    else:
        img = 255 * img.astype("uint8")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
    cv2.imwrite(path, img)
    
# Define a function for undistortion raw images
def distortion_correction(img, objp, imgp):
    # gray.shape[::-1] returns the iamge size(hight x width)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objp, imgp, gray.shape[::-1], None, None)
    undst_corr = cv2.undistort(img, mtx, dist, None, mtx)
    return undst_corr

# Test distortion correction on calibration image
camera_test_image = read_image("./camera_cal/calibration2.jpg")
undist_camera_test_image = distortion_correction(camera_test_image, objpoints, imgpoints)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 7))
ax1.imshow(camera_test_image)
ax1.set_title("Chess Board Raw Image", fontsize=15)
ax2.imshow(undist_camera_test_image)
ax2.set_title("Chess Board Distortion Correction Image", fontsize=15)

# Test undistortion on test image
test_img = read_image("./test_images/test2.jpg")
undist_test_img = distortion_correction(test_img, objpoints, imgpoints)


plt.figure(figsize=(12, 7))
plt.subplot(2, 2, 1)
plt.imshow(test_img)
plt.title("Raw Test Image")
plt.subplot(2, 2, 2)
plt.imshow(undist_test_img)
plt.title("Distortion Test Correction")

write_image("output_images/undist_test_image.jpg", undist_test_img)

print("Finished undistorted test image")

Finished undistorted test image


## Use color transforms, gradients, etc., to create a thresholded binary image.

In [67]:
# Define a function that applies Sobel x or y, 
# then takes an absolute value and applies a threshold.
# Note: calling your function with orient='x', thresh_min=5, thresh_max=100
# Calculate directional gradient
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    # 3) Take the absolute value of the derivative or gradient
    x = (orient == 'x')
    y = (orient == 'y')
    abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, x, y))
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return grad_binary

# Define a function that applies Sobel x and y, 
# then computes the magnitude of the gradient
# and applies a threshold
# Calculate gradient magnitude
def mag_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1
    return mag_binary

# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
# Calculate gradient direction
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.sqrt(sobelx**2)
    abs_sobely = np.sqrt(sobely**2)
    abssobel = np.sqrt(sobelx**2 + sobely**2)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    absgraddir = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    dir_binary = np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return dir_binary

# Define a function that thresholds the S-channel of HLS
# Use exclusive lower bound (>) and inclusive upper (<=)
def hls_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    # 2) Apply a threshold to the S channel
    s_channel = hls[:,:,2]
    # 3) Return a binary image of threshold result
    color_binary = np.zeros_like(s_channel)
    color_binary[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    #binary_output = np.copy(img) # placeholder line
    return color_binary

# Choose a Sobel kernel size
ksize = 5 # Choose a larger odd number to smooth gradient measurements

def get_combined_sobel(img):
    gradx = abs_sobel_thresh(img, orient='x', sobel_kernel=ksize, thresh=(60, 120))
    grady = abs_sobel_thresh(img, orient='y', sobel_kernel=ksize, thresh=(20, 50))
    mag_binary = mag_thresh(img, sobel_kernel=ksize, thresh=(30, 500))
    dir_binary = dir_threshold(img, sobel_kernel=ksize, thresh=(0.7, 1.3))
    combined = np.zeros_like(gradx)
    combined[((gradx >= 1) & (grady >= 1)) | ((mag_binary >= 1) & (dir_binary >= 1))] = 1
    return combined

# Test distortion correction on test image
test_image = read_image("./test_images/test5.jpg")
undist_test_image = distortion_correction(test_image, objpoints, imgpoints)
color_binary = hls_select(undist_test_image,thresh=(170,255))
combined_binary = get_combined_sobel(undist_test_image)
result = cv2.bitwise_or(combined_binary, color_binary)


plt.figure(figsize=(12, 7))
plt.subplot(2, 2, 1)
plt.imshow(test_image)
plt.title("Raw Image")
print(len(test_image.shape))

plt.subplot(2, 2, 2)
plt.imshow(combined_binary, cmap="gray")
plt.title("Combined Gradiant Threshold Binary")
#write_image("output_images/combined_gradiant_threshold_binary.jpg", combined_binary)
plt.imsave("output_images/combined_gradiant_threshold_binary.jpg", combined_binary, cmap="gray")

plt.subplot(2, 2, 3)
plt.imshow(color_binary)
plt.title("Color Threshold binary")
#write_image("output_images/color_threshold_binary.jpg", color_binary)
plt.imsave("output_images/color_threshold_binary.jpg", color_binary)

plt.subplot(2, 2, 4)
plt.imshow(result, cmap="gray")
print(len(result.shape))
plt.title("Combined Color and Gradiant Thresholds")
#write_image("output_images/combined_color_and_gradiant_threshold_binary.jpg", result)
plt.imsave("output_images/combined_color_and_gradiant_threshold_binary.jpg", result, cmap="gray")

print("Finished threshold binary image")

3
2
Finished threshold binary image


## Apply a perspective transform to rectify binary image ("birds-eye view")

In [69]:
def warped_image(combined_image, orient='M'):
    image_size = (combined_image.shape[1], combined_image.shape[0])
    
    src = np.float32(
    [[730, 460],    
     [1100, 690],   
     [200, 695],    
     [600, 455]])   
    dst = np.float32(
    [[1120, 0],      # top right
     [1120, 680],    # bottom right
     [200, 680],     # bottom left
     [200, 0]])      # top left  

    if orient == 'M':
        M = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(combined_image, M, image_size, flags=cv2.INTER_LINEAR)
    if orient == 'Minv':
        Minv = cv2.getPerspectiveTransform(dst, src)
        warped = cv2.warpPerspective(combined_image, Minv, image_size, flags=cv2.INTER_LINEAR)
    return warped
    
hls_binary = hls_select(undist_test_image, thresh=(90, 255))
warped_binary = warped_image(result, orient='M')

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8))
ax1.imshow(hls_binary, cmap='gray')
ax1.plot(730, 460, '.')    # top right
ax1.plot(1100, 690, '.')   # bottom right
ax1.plot(200, 695, '.')    # bottom left
ax1.plot(600, 455, '.')    # top left
ax1.set_title('HLS Select Image', fontsize=25)
ax2.imshow(warped_binary, cmap="gray")
ax2.set_title('Warped Image', fontsize=25)

# Take a histogram of the bottom half of the image
histogram = np.sum(warped_binary[warped_binary.shape[0]//2:,:], axis=0)
plt.figure(figsize=(16,8))
plt.plot(histogram)
plt.show()


## Test pipeline on video

In [42]:
def measure_curvature(leftx, lefty, rightx, righty, ploty, image_size):
    y_eval = np.max(ploty)
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # Now our radius of curvature is in meters
    # Example values: 632.1 m    626.2 m
    
    # Calculate Lane Deviation from center of lane:
    # First we calculate the intercept points at the bottom of our image, then use those to 
    # calculate the lane deviation of the vehicle (assuming camera is in center of vehicle)
    scene_height = image_size[0] * ym_per_pix
    scene_width = image_size[1] * xm_per_pix
    
    left_intercept = left_fit_cr[0] * scene_height ** 2 + left_fit_cr[1] * scene_height + left_fit_cr[2]
    right_intercept = right_fit_cr[0] * scene_height ** 2 + right_fit_cr[1] * scene_height + right_fit_cr[2]
    calculated_center = (left_intercept + right_intercept) / 2.0
    
    lane_deviation = (calculated_center - scene_width / 2.0)
    
    return left_curverad, right_curverad, lane_deviation

def locate_lane_lines(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)

    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    left_curvature, right_curvature, lane_deviation = measure_curvature(leftx, lefty, rightx, righty, ploty, out_img.shape)
    
    return left_fitx, right_fitx, ploty, left_fit, right_fit, out_img, left_curvature, right_curvature, lane_deviation

In [43]:
# Visualization
# At this point, you're done! But here is how you can visualize the result as well:
left_fitx, right_fitx, ploty, left_fit, right_fit, out_img, left_radius, right_radius, lane_deviation = locate_lane_lines(warped_binary)
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)


(720, 0)

In [44]:
binary_warped = warped_binary
# Assume you now have a new warped binary image 
# from the next frame of video (also called "binary_warped")
# It's now much easier to find line pixels!
nonzero = binary_warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
margin = 100
left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

# Again, extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds]
# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)
# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

In [45]:
# Create an image to draw on and an image to show the selection window
out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
window_img = np.zeros_like(out_img)
# Color in left and right line pixels
out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))
right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

(720, 0)

In [51]:

#def draw_lanes(binary_warped, undistorted_img, Minv, left_fitx, right_fitx, ploty, left_radius, right_radius, lane_deviation):    
def draw_lanes(binary_warped, undistorted_img, left_fitx, right_fitx, ploty, left_radius, right_radius, lane_deviation):    
    # Create a blank image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    #newwarp = cv2.warpPerspective(color_warp, Minv, (undistorted_img.shape[1], undistorted_img.shape[0])) 
    newwarp = warped_image(color_warp, orient="Minv")
    
    # Combine the result with the original image
    result = cv2.addWeighted(undistorted_img, 1, newwarp, 0.3, 0)
    
    curvature_text = "Curvature: Left = " + str(np.round(left_radius, 2)) + ", Right = " + str(np.round(right_radius, 2)) 
    font = cv2.FONT_HERSHEY_SIMPLEX    
    cv2.putText(result, curvature_text, (30, 60), font, 1, (255,255,255), 2)
    deviation_text = "Lane deviation from center = {:.2f} m".format(lane_deviation) 
    font = cv2.FONT_HERSHEY_SIMPLEX    
    cv2.putText(result, deviation_text, (30, 90), font, 1, (255,255,255), 2)
        
    return result

left_curvature, right_curvature, lane_deviation = measure_curvature(leftx, lefty, rightx, righty, ploty, binary_warped.shape)
result = draw_lanes(binary_warped, undist_test_image, left_fitx, right_fitx, ploty, left_curvature, right_curvature, lane_deviation)
plt.figure(figsize=(16,8))
plt.imshow(result)
plt.axis("off");
write_image("output_images/result.jpg", result)



In [52]:
def process_video(image):
    """
    Execute our image processing pipeline on the provided image.
    """    
    undist_test_image = distortion_correction(image, objpoints, imgpoints)
    color_binary = hls_select(undist_test_image,thresh=(170,255))
    combined_binary = get_combined_sobel(undist_test_image)
    combined_binary = cv2.bitwise_or(combined_binary, color_binary)
    binary_warped = warped_image(combined_binary)
    left_fitx, right_fitx, ploty, left_fit, right_fit, out_img, left_curvature, right_curvature, lane_deviation = locate_lane_lines(binary_warped)
    #left_curvature, right_curvature, lane_deviation = measure_curvature(leftx, lefty, rightx, righty, ploty, warped_binary.shape)
    #lane_lines = draw_lanes(binary_warped, undist_test_image, left_fitx, right_fitx, ploty, left_curvature, right_curvature, lane_deviation)
    #left_curvature, right_curvature, lane_deviation = measure_curvature(leftx, lefty, rightx, righty, ploty, out_img.shape)
    #print(left_curvature, right_curvature, lane_deviation)
    lane_lines = draw_lanes(binary_warped, undist_test_image, left_fitx, right_fitx, ploty, left_curvature, right_curvature, lane_deviation)
    return lane_lines

In [53]:

# Run on a test image
img = cv2.imread("test_images/test2.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

result = process_video(img)

plt.figure(figsize=(16,8))
plt.imshow(result)
plt.axis("off");


In [54]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

p4_video_output = "output_images/project_video_output.mp4"
clip1 = VideoFileClip("project_video.mp4");
clip1_output = clip1.fl_image(process_video) #NOTE: this function expects color images!!
%time clip1_output.write_videofile(p4_video_output, audio=False)

[MoviePy] >>>> Building video output_images/project_video_output.mp4
[MoviePy] Writing video output_images/project_video_output.mp4


100%|█████████▉| 1260/1261 [16:55<00:00,  1.24it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project_video_output.mp4 

CPU times: user 16min 54s, sys: 1min 39s, total: 18min 34s
Wall time: 16min 56s


In [55]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(p4_video_output))

In [70]:
video_output = "challenge_video_output.mp4"
clip1 = VideoFileClip("challenge_video.mp4");
clip1_output = clip1.fl_image(process_video) #NOTE: this function expects color images!!
%time clip1_output.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video challenge_video_output.mp4
[MoviePy] Writing video challenge_video_output.mp4


100%|██████████| 485/485 [06:35<00:00,  1.25it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_output.mp4 

CPU times: user 6min 27s, sys: 39.8 s, total: 7min 7s
Wall time: 6min 36s


In [71]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))